# Import related pacakages

In [5]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

In [2]:
import pandas as pd

infile='./analysis/Summary_of_different_model_results.csv'
model_data=pd.read_csv(infile,index_col=0)

svg_need_data_df=pd.DataFrame()

for index, row in model_data.iterrows():
    mw_cb=str(row['reaction_mw_not_condider_subunit'])+' # '+str(row['reaction_mw_condider_subunit'])
    kcat_cb=str(round(row['smoment_no_adj_kcat'],2))+' # '+str(round(row['smoment_adj_kcat'],2))+' # '+\
        str(round(row['kcat_in_vitro'],2))+' # '+str(round(row['kapp'],2))+' # '+str(round(row['kcat_zhaoxin'],2))\
        +' # '+str(round(row['kcat_GO'],2))+' # '+str(round(row['kcat_ec_model'],2))
    ori_gecko_smoment_flux_cb=str(round(row['model_ori_fluxes'],2))+' # '+str(round(row['model_gecko_adj_fluxes'],2))+\
        ' # '+str(round(row['model_smoment_adj_fluxes'],2))
    ori_gecko_smoment_subunit_flux_cb=str(round(row['model_ori_fluxes'],2))+' # '+str(round(row['ECMpy_fluxes'],2))+\
        ' # '+str(round(row['model_gecko_adj_subunit_fluxes'],2))+' # '+str(round(row['model_smoment_adj_subunit_fluxes'],2))
    svg_need_data_df.loc[index,'ori_gecko_smoment_flux_cb']=ori_gecko_smoment_flux_cb
    svg_need_data_df.loc[index,'ori_gecko_smoment_subunit_flux_cb']=ori_gecko_smoment_subunit_flux_cb
    svg_need_data_df.loc[index,'mw_cb']=mw_cb
    svg_need_data_df.loc[index,'kcat_cb']=kcat_cb
svg_need_data_df.to_csv('./analysis/svg_need_data_df.csv', header=True, index=True,mode='w')

In [7]:
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_mw.svg"
rclass=['st6','st15','st18'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cvg_need_data_df,'mw_cb',insvg,outsvg,rclass)

In [8]:
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_kcat.svg"
rclass=['st6','st15','st18'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cvg_need_data_df,'kcat_cb',insvg,outsvg,rclass)

In [9]:
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_ori_gecko_smoment_flux_cb.svg"
rclass=['st6','st15','st18'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cvg_need_data_df,'ori_gecko_smoment_flux_cb',insvg,outsvg,rclass)

In [10]:
insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_ori_gecko_smoment_subunit_flux_cb.svg"
rclass=['st6','st15','st18'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cvg_need_data_df,'ori_gecko_smoment_subunit_flux_cb',insvg,outsvg,rclass)

In [11]:
import pandas as pd
import re
import numpy as np
import math

ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_kappori_solution_df_pfba.csv',index_col=0)
ECMpy_adj_round1_solution_frame=pd.read_csv('./analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv',index_col=0)
Orimodel_solution_frame=pd.read_csv('./analysis/Orimodel_solution_df_pfba.csv',index_col=0)

model_data=pd.DataFrame()
for eachreaction in ECMpy_solution_frame.index:
    model_data.loc[eachreaction,'model_ori_fluxes']=Orimodel_solution_frame.loc[eachreaction,'fluxes']
    model_data.loc[eachreaction,'ECMpy_fluxes']=ECMpy_solution_frame.loc[eachreaction,'fluxes']
    model_data.loc[eachreaction,'ECMpy_adj_round1_fluxes']=ECMpy_adj_round1_solution_frame.loc[eachreaction,'fluxes']  
model_data.to_csv('./analysis/iML1515_model_cb_1.csv')

cb_df=pd.DataFrame()
for index, row in model_data.iterrows():
    index_reverse=index+'_reverse'
    if re.search('reverse',index):
        pass
    elif index_reverse in model_data.index:
        model_ori_reverse_fluxes=float(model_data.loc[index_reverse,'model_ori_fluxes'])
        ECMpy_reverse_fluxes=float(model_data.loc[index_reverse,'ECMpy_fluxes'])
        ECMpy_adj_round1_reverse_fluxes=float(model_data.loc[index_reverse,'ECMpy_adj_round1_fluxes'])

        if math.isnan(model_ori_reverse_fluxes):
            model_ori_fluxes=model_data.loc[index,'model_ori_fluxes']
        else:
            model_ori_fluxes=np.max([model_data.loc[index,'model_ori_fluxes'],model_data.loc[index_reverse,'model_ori_fluxes']])
            
        if math.isnan(ECMpy_reverse_fluxes):
            ECMpy_fluxes=model_data.loc[index,'ECMpy_fluxes']
        else:
            ECMpy_fluxes=np.max([model_data.loc[index,'ECMpy_fluxes'],model_data.loc[index_reverse,'ECMpy_fluxes']])
            
        if math.isnan(ECMpy_adj_round1_reverse_fluxes):
            ECMpy_adj_round1_fluxes=model_data.loc[index,'ECMpy_adj_round1_fluxes']
        else:
            ECMpy_adj_round1_fluxes=np.max([model_data.loc[index,'ECMpy_adj_round1_fluxes'],model_data.loc[index_reverse,'ECMpy_adj_round1_fluxes']])
                        
        flux_cb=str(round(model_ori_fluxes,2))+' # '+' # '+str(round(ECMpy_fluxes,2))+' # ' \
    +' # '+str(round(ECMpy_adj_round1_fluxes,2))
        cb_df.loc[index,'flux_cb']=flux_cb
    else:
        flux_cb=str(round(row['model_ori_fluxes'],2))+' # '+' # '+str(round(row['ECMpy_fluxes'],2))+' # ' \
        +' # '+str(round(row['ECMpy_adj_round1_fluxes'],2))
        cb_df.loc[index,'flux_cb']=flux_cb
cb_df.to_csv('./analysis/iML1515_model_cb_1_map.csv', header=True, index=True,mode='w')

insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_fluxes_1.svg"
rclass=['st6','st15','st18','st16'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cb_df,'flux_cb',insvg,outsvg,rclass)

In [12]:
import pandas as pd
import re
import numpy as np
import math

ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_kappori_solution_df_pfba.csv',index_col=0)
ECMpy_adj_round1_solution_frame=pd.read_csv('./analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv',index_col=0)
Orimodel_solution_frame=pd.read_csv('./analysis/Orimodel_solution_df_pfba.csv',index_col=0)

model_data=pd.DataFrame()
for eachreaction in ECMpy_solution_frame.index:
    model_data.loc[eachreaction,'model_ori_fluxes']=Orimodel_solution_frame.loc[eachreaction,'fluxes']
    model_data.loc[eachreaction,'ECMpy_fluxes']=ECMpy_solution_frame.loc[eachreaction,'fluxes']
    model_data.loc[eachreaction,'ECMpy_adj_round1_fluxes']=ECMpy_adj_round1_solution_frame.loc[eachreaction,'fluxes']  
model_data.to_csv('./analysis/iML1515_model_cb_2.csv')

cb_df=pd.DataFrame()
for index, row in model_data.iterrows():
    index_reverse=index+'_reverse'
    if re.search('reverse',index):
        pass
    elif index_reverse in model_data.index:
        model_ori_reverse_fluxes=float(model_data.loc[index_reverse,'model_ori_fluxes'])
        ECMpy_reverse_fluxes=float(model_data.loc[index_reverse,'ECMpy_fluxes'])
        ECMpy_adj_round1_reverse_fluxes=float(model_data.loc[index_reverse,'ECMpy_adj_round1_fluxes'])

        if math.isnan(model_ori_reverse_fluxes):
            model_ori_fluxes=model_data.loc[index,'model_ori_fluxes']
        else:
            model_ori_fluxes=np.max([model_data.loc[index,'model_ori_fluxes'],model_data.loc[index_reverse,'model_ori_fluxes']])
            
        if math.isnan(ECMpy_reverse_fluxes):
            ECMpy_fluxes=model_data.loc[index,'ECMpy_fluxes']
        else:
            ECMpy_fluxes=np.max([model_data.loc[index,'ECMpy_fluxes'],model_data.loc[index_reverse,'ECMpy_fluxes']])
            
        if math.isnan(ECMpy_adj_round1_reverse_fluxes):
            ECMpy_adj_round1_fluxes=model_data.loc[index,'ECMpy_adj_round1_fluxes']
        else:
            ECMpy_adj_round1_fluxes=np.max([model_data.loc[index,'ECMpy_adj_round1_fluxes'],model_data.loc[index_reverse,'ECMpy_adj_round1_fluxes']])
                        
        flux_cb=str(round(model_ori_fluxes,2))+' # '+' # '+str(round(ECMpy_fluxes,2))+' # ' \
    +' # '+str(round(ECMpy_adj_round1_fluxes,2))
        cb_df.loc[index,'flux_cb']=flux_cb
    else:
        flux_cb=str(round(row['model_ori_fluxes'],2))+' # '+' # '+str(round(row['ECMpy_fluxes'],2))+' # ' \
        +' # '+str(round(row['ECMpy_adj_round1_fluxes'],2))
        cb_df.loc[index,'flux_cb']=flux_cb
cb_df.to_csv('./analysis/iML1515_model_cb_2_map.csv', header=True, index=True,mode='w')

insvg="./data/Central_metabolic_pathway.svg"
outsvg="./analysis/Central_metabolic_pathway_map_fluxes_2.svg"
rclass=['st6','st15','st18','st16'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cb_df,'flux_cb',insvg,outsvg,rclass)

In [2]:
import cobra
import pandas as pd
model_gecko_adj_subunit=cobra.io.read_sbml_model( "./data/iML1515_GECKO_adj_subunit.xml")
model_smoment_adj_subunit=cobra.io.read_sbml_model( "./data/iML1515_smoment_adj_subunit.xml")

model_gecko_adj_subunit.reactions.get_by_id("ER_pool_TG_").upper_bound=0.228#0.095
model_smoment_adj_subunit.reactions.get_by_id("ER_pool_TG_").upper_bound=0.228#0.095
model_gecko_adj_subunit_solution = cobra.flux_analysis.pfba(model_gecko_adj_subunit)
model_gecko_adj_subunit_solution_frame = model_gecko_adj_subunit_solution.to_frame()
model_gecko_adj_subunit_solution_frame.to_csv('./analysis/model_gecko_adj_subunit_solution_pfba.csv')
model_smoment_adj_subunit_solution = cobra.flux_analysis.pfba(model_smoment_adj_subunit)
model_smoment_adj_subunit_solution_frame = model_smoment_adj_subunit_solution.to_frame()
model_smoment_adj_subunit_solution_frame.to_csv('./analysis/model_smoment_adj_subunit_solution_pfba.csv')

ECMpy_solution_frame=pd.read_csv('./analysis/ECMpy_kappori_solution_df_pfba.csv',index_col=0)
Orimodel_solution_frame=pd.read_csv('./analysis/Orimodel_solution_df_pfba.csv',index_col=0)

In [ ]:
import pandas as pd
import re
import numpy as np
total_solution_frame=pd.DataFrame()

for eachreaction in ECMpy_solution_frame.index:
    total_solution_frame.loc[eachreaction,'reaction']=eachreaction
    total_solution_frame.loc[eachreaction,'model_ori_fluxes']=Orimodel_solution_frame.loc[eachreaction,'fluxes']
    total_solution_frame.loc[eachreaction,'ECMpy_fluxes']=ECMpy_solution_frame.loc[eachreaction,'fluxes']
    
    gecko_list=[]
    for reaction in model_gecko_adj_subunit_solution_frame.index:
        search_reaction=eachreaction+'_'
        if eachreaction==reaction:
            gecko_list.append(model_gecko_adj_subunit_solution_frame.loc[reaction,'fluxes'])
        elif re.search(search_reaction,reaction):
            gecko_list.append(model_gecko_adj_subunit_solution_frame.loc[reaction,'fluxes'])   
    #print(eachreaction,gecko_list)
    if len(gecko_list)>0:
        total_solution_frame.loc[eachreaction,'model_gecko_adj_subunit_fluxes']=np.max(gecko_list)

    smoment_list=[]
    for reaction in model_smoment_adj_subunit_solution_frame.index:
        search_reaction=eachreaction+'_'
        if eachreaction==reaction:
            smoment_list.append(model_smoment_adj_subunit_solution_frame.loc[reaction,'fluxes'])
        elif re.search(search_reaction,reaction):
            smoment_list.append(model_smoment_adj_subunit_solution_frame.loc[reaction,'fluxes'])
    if len(smoment_list)>0:        
        total_solution_frame.loc[eachreaction,'model_smoment_adj_subunit_fluxes']=np.max(smoment_list)    
    
total_solution_frame.to_csv('./analysis/iML1515_20201103.csv')

In [ ]:
import pandas as pd
import numpy as np
import math

infile='./analysis/iML1515_diff_model_cb_20201103.csv'
model_data=pd.read_csv(infile,index_col=0)
cb_df=pd.DataFrame()

for index, row in model_data.iterrows():
    index_reverse=index+'_reverse'
    if re.search('reverse',index):
        pass
    elif index_reverse in model_data.index:
        model_ori_reverse_fluxes=float(model_data.loc[index_reverse,'model_ori_fluxes'])
        ECMpy_reverse_fluxes=float(model_data.loc[index_reverse,'ECMpy_fluxes'])
        model_gecko_adj_subunit_reverse_fluxes=float(model_data.loc[index_reverse,'model_gecko_adj_subunit_fluxes'])
        model_smoment_adj_subunit_reverse_fluxes=float(model_data.loc[index_reverse,'model_smoment_adj_subunit_fluxes'])
        
        if math.isnan(model_ori_reverse_fluxes):
            model_ori_fluxes=model_data.loc[index,'model_ori_fluxes']
        else:
            model_ori_fluxes=np.max([model_data.loc[index,'model_ori_fluxes'],model_data.loc[index_reverse,'model_ori_fluxes']])
            
        if math.isnan(ECMpy_reverse_fluxes):
            ECMpy_fluxes=model_data.loc[index,'ECMpy_fluxes']
        else:
            ECMpy_fluxes=np.max([model_data.loc[index,'ECMpy_fluxes'],model_data.loc[index_reverse,'ECMpy_fluxes']])
            
        if math.isnan(model_gecko_adj_subunit_reverse_fluxes):
            model_gecko_adj_subunit_fluxes=model_data.loc[index,'model_gecko_adj_subunit_fluxes']
        else:
            model_gecko_adj_subunit_fluxes=np.max([model_data.loc[index,'model_gecko_adj_subunit_fluxes'],\
                                                   model_data.loc[index_reverse,'model_gecko_adj_subunit_fluxes']])
            
        if math.isnan(model_smoment_adj_subunit_reverse_fluxes):
            model_smoment_adj_subunit_fluxes=model_data.loc[index,'model_smoment_adj_subunit_fluxes']
        else:
            model_smoment_adj_subunit_fluxes=np.max([model_data.loc[index,'model_smoment_adj_subunit_fluxes'],\
                                                     model_data.loc[index_reverse,'model_smoment_adj_subunit_fluxes']])
        flux_cb=str(round(model_ori_fluxes,2))+' # '+' # '+str(round(ECMpy_fluxes,2))+' # ' \
    +' # '+str(round(model_gecko_adj_subunit_fluxes,2))+' # '+str(round(model_smoment_adj_subunit_fluxes,2))
        cb_df.loc[index,'flux_cb2']=flux_cb
    else:
        flux_cb=str(round(row['model_ori_fluxes'],2))+' # '+' # '+str(round(row['ECMpy_fluxes'],2))+' # ' \
        +' # '+str(round(row['model_gecko_adj_subunit_fluxes'],2))+' # '+str(round(row['model_smoment_adj_subunit_fluxes'],2))
        cb_df.loc[index,'flux_cb2']=flux_cb

cb_df.to_csv('./cb_df2.csv', header=True, index=True,mode='w')

In [ ]:
insvg="../AutoEnzModel/flux_cb/中心代谢途径.svg"
outsvg="./中心代谢途径_map_fluxes.svg"
rclass=['st6','st14','st18','st17'] #svg文件中反应名称的CSS类, 可能有多个对应上下调#此处可更换svg文件中反应ID的分类关系

draw_svg(cb_df,'flux_cb2',insvg,outsvg,rclass)